# Ctypes

A way to implement C code in python!

In [67]:
import ctypes
import os

### compiling for c sharp: gcc -shared -fPIC -o functionslib.so functions.c

### Load the shared library

In [68]:
path = os.getcwd()
functionslib = ctypes.CDLL(os.path.join(path, 'functionslib.so'))

In [69]:
functionslib.add2.argtypes = [ctypes.c_int]
functionslib.add2.restype = ctypes.c_int
functionslib.add2(2)

4

Windows loading of library

In [70]:
#functionslib = ctypes.WinDLL("C:\\wsl\\final presentation\\project-gdisalvo30\\functionslib.dll")

## We can also make code work from c++!

In [71]:
cpluspluslib = ctypes.CDLL(os.path.join(path, 'ccpluslib.so'))

In [72]:
cpluspluslib.multiplybythree.argtypes = [ctypes.c_int]
cpluspluslib.multiplybythree.restype = ctypes.c_int
cpluspluslib.multiplybythree(3)

9

### Must include extern "C" {} in order for C++ code to work within the C++, it is okay to use the same ctypes

### compile with: g++ -fPIC -shared -o ccpluslib.so cplusfunctions.cpp

### making a struct for our online mean function to be used by our code in C

In [73]:
class OnlineMeanVar(ctypes.Structure):
    _fields_ = [("meanA", ctypes.c_double),
               ("vari", ctypes.c_double),
               ("countof", ctypes.c_double),
               ("M2", ctypes.c_double)]

instantiating our variable

In [74]:
omv = OnlineMeanVar(0.0, 0.0, 0.0, 0.0)

In [75]:
functionslib.Update.argtypes = [ctypes.c_double]
functionslib.Update.restype = ctypes.POINTER(OnlineMeanVar)

Bug here, not sure why but cannot perform more than one update? Kernel crash.

In [76]:
omv = functionslib.Update(1, omv)

In [77]:
#omv = functionslib.Update(0, omv)

In [78]:
omv.contents.meanA

1.0

Here is python version of it, (was thinking of comparing run times after I was done ironing the bug)

In [79]:
import numpy as np
class OnlineMeanVar2():
    def __init__(self):
        self.meanA = 0.0
        self.vari = 0.0
        self.countof = 0
        self.M2 = 0.0
    
    def update(self, newValue):
                self.countof += 1
                delta = newValue - self.meanA
                self.meanA += delta / self.countof
                delta2 = newValue - self.meanA
                self.M2 += delta * delta2
                
    def mean(self):
        return self.meanA
    
    def var(self):
        return (self.M2 / self.countof)
        
    def count(self):
        return self.countof

In [80]:
omv2 = OnlineMeanVar2()
omv2.update(1)

In [81]:
omv2.meanA

1.0

## Real Performance Numbers

In [82]:
import timeit
import time

In [83]:
functionslib.multiplicativescience.restype = ctypes.c_double
somenumber = functionslib.multiplicativescience()

In [84]:
somenumber

45015000.0

Number in timeit function is how many times to run the function (both are 100 million to emphasize difference of time but are still relative in smaller amount of times ran)

In [90]:
csharptime = (timeit.Timer(functionslib.multiplicativescience).timeit(number=100000000))

making the same function in python

In [86]:
def multiplicativepython():
    x = 0.0
    while x < 10000000:
        x = x + 5
        x = x * 1000
        x = x * 3
    return x

In [88]:
anothernumber = multiplicativepython()
anothernumber

45015000.0

In [91]:
pythontime = (timeit.Timer(multiplicativepython).timeit(number=100000000))

Code in C runtime:

In [92]:
csharptime

6.733832755000549

Code in Python runtime:

In [93]:
pythontime

12.889303706000646

In [94]:
pythontime / csharptime

1.9141110530891996

This means that the code that was written in C, was roughly 191% faster in comparison to the code that was written in python.